# Dados

In [0]:
database = [
    {"classe": "horario", "sentencas": (
        "qual vai ser o horário de robótica?",
        "Alguém que faz IC com Abner(circuitos) tem como perguntar a ele qual o horário?",
        "Alguém sabe o horário das disciplinas optativas?",
        "Robótica esse período, foi em quais horários?",
        "Qual horário de calculo 2 nesse período?",
        "Qual o horário de IP2", "Que horas começa a aula de TC?"
    )},
    {"classe": "duvida","sentencas": (
        "O que você pode fazer?", "O que você faz?", "Em que você pode me ajudar?", 
        "Me diga o que pode fazer", "Qual sua função?", "para que você serve?", "Do que você é capaz?"
    )},
    {"classe": "ementa", "sentencas": (
        "Tem como enviar ementa da disciplina?", "Ementa de teoria", 
        "Qual a ementa de algoritmos", "Assunto da cadeira de algebra", 
        "Plano de aula de arquitetura", "Assunto das aulas de compiladores"
    )},
    {"classe": "pre_requisito", "sentencas": (
        "Sabe se precisa de algum pré requisito para IA?", "SO tem pre requisito?", 
        "Qual pre-requisito para prog2", "Lista de pre requisitos para projetão",
        "quais a cadeiras que preciso ter pago para pagar Sistemas Operacionais", 
        "Quais os pre-requisitos", "o que tenho que pagar antes de banco de dados?"
    )},
    {"classe": "carga_horária", "sentencas": (
        "Sem carga horária não serve.",
        "Ele disse que cumpriu a carga horária",
        "Se der certificado e lá tiver escrito a carga horária",
        "Qual a carga horária", "Quanto tempo", "Qual a ch?", "Quantas horas"
    )},
    {"classe": "sala", "sentencas": (
        "discreta I qual a sala?",
        "Alguém sabe a sala de circuitos?",
        "qual a sala de Prog2", "onde é a aula de calculo?",
    )},
    {"classe": "professor", "sentencas": (
        "Quem ta dando discreta?", "qual professor de pensamento computacional",
        "quem eh o professor de IP2", "quem dá matematica discreta", 
        "qual o professor de algebra", "qual o nome do professor de prog2"
    )},
    {"classe": "estagio", "sentencas": (
        "Como funciona eso", "como cortar eso", "trabalho dispensa eso", "estagio corta eso",
        "como posso dispensar ESO", "trabalho carteira asinada dispensa estágio"
    )}
]

In [0]:
from random import shuffle
def train_test_split(data, test_size=.3):
    train = []
    test = []
    for classe in data:
        s = list(classe['sentencas'])
        shuffle(s)
        train.append({'classe': classe['classe'], 'sentencas': s[int(len(classe['sentencas'])*test_size):]})
        test.append({'classe': classe['classe'], 'sentencas': s[0:int(len(classe['sentencas'])*test_size)]})
    return train, test

training_data, test_data = train_test_split(database)

classes = list(set([a['classe'] for a in training_data]))
classes

['pre_requisito',
 'sala',
 'duvida',
 'horario',
 'estagio',
 'ementa',
 'carga_horária',
 'professor']

# Naive Bayes + Bag of Words

In [0]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
corpus_words = {}
class_words = {}

for c in classes:
    class_words[c] = []

for data in training_data:
    for sentence in data['sentencas']:
        for word in nltk.word_tokenize(sentence):
            if word not in ["?", "'s"]:
                stemmed_word = stemmer.stem(word.lower())
                if stemmed_word not in corpus_words:
                    corpus_words[stemmed_word] = 1
                else:
                    corpus_words[stemmed_word] += 1
                class_words[data['classe']].extend([stemmed_word])

print ("Corpus words and counts: %s \n" % corpus_words)
print ("Class words: %s" % class_words)

Corpus words and counts: {'qual': 10, 'vai': 1, 'ser': 1, 'o': 7, 'horário': 4, 'de': 17, 'robótic': 2, 'que': 9, 'hora': 1, 'começ': 1, 'a': 9, 'aul': 2, 'tc': 1, 'ip2': 2, 'calculo': 1, '2': 1, 'ness': 1, 'período': 2, 'ess': 1, ',': 1, 'foi': 1, 'em': 1, 'qua': 3, 'você': 3, 'pod': 2, 'faz': 2, 'me': 1, 'dig': 1, 'par': 3, 'serv': 2, 'do': 1, 'é': 1, 'capaz': 1, 'sua': 1, 'função': 1, 'plano': 1, 'arquitetur': 1, 'assunto': 1, 'das': 1, 'aula': 1, 'compilad': 1, 'ement': 3, 'algoritmo': 1, 'tem': 2, 'como': 3, 'envi': 1, 'da': 1, 'disciplin': 1, 'teor': 1, 'os': 1, 'pre-requisitos': 1, 'list': 1, 'pre': 2, 'requisito': 2, 'projetão': 1, 'tenho': 1, 'pag': 2, 'ant': 1, 'banco': 1, 'dado': 1, 'cadeira': 1, 'preciso': 1, 'ter': 1, 'pago': 1, 'sistema': 1, 'operaciona': 1, 'so': 1, 'el': 1, 'diss': 1, 'cumpriu': 1, 'carg': 3, 'horár': 3, 'se': 1, 'der': 1, 'certificado': 1, 'e': 1, 'lá': 1, 'tiv': 1, 'escrito': 1, 'ch': 1, 'sem': 1, 'não': 1, '.': 1, 'quanto': 1, 'tempo': 1, 'sal': 3, '

In [0]:
def calculate_class_score(sentence, class_name, show_details=True):
    score = 0
    for word in nltk.word_tokenize(sentence):
        if stemmer.stem(word.lower()) in class_words[class_name]:
            score += 1
            if show_details:
                print ("   match: %s" % stemmer.stem(word.lower()))
    return score

In [0]:
sentence = "Qual a sala de psicologia2?"

for c in class_words.keys():
    print ("Class: %s  Score: %s \n" % (c, calculate_class_score(sentence, c)))

   match: a
   match: de
Class: pre_requisito  Score: 2 

   match: qual
   match: a
   match: sal
   match: de
Class: sala  Score: 4 

   match: qual
Class: duvida  Score: 1 

   match: qual
   match: a
   match: de
Class: horario  Score: 3 

Class: estagio  Score: 0 

   match: qual
   match: a
   match: de
Class: ementa  Score: 3 

   match: qual
   match: a
Class: carga_horária  Score: 2 

   match: qual
   match: de
Class: professor  Score: 2 



In [0]:
def calculate_class_score(sentence, class_name, show_details=True):
    score = 0
    for word in nltk.word_tokenize(sentence):
        if stemmer.stem(word.lower()) in class_words[class_name]:
            score += (1 / corpus_words[stemmer.stem(word.lower())])
            if show_details:
                print ("   match: %s (%s)" % (stemmer.stem(word.lower()), 1 / corpus_words[stemmer.stem(word.lower())]))
    return score

In [0]:
def classify(sentence):
    high_class = None
    high_score = 0
    for c in class_words.keys():
        score = calculate_class_score(sentence, c, show_details=False)
        if score > high_score:
            high_class = c
            high_score = score

    return high_class, high_score

In [0]:
to_test = []
for classe in test_data:
    for sentenca in classe['sentencas']:
        to_test.append((classe['classe'], sentenca))

matriz = [[0 for c in range(len(classes))] for l in range(len(classes))]

corretas, todas = 0, 0
for label, sentence in to_test:
    predict, confidence = classify(sentence)
    matriz[classes.index(label)][classes.index(predict)] += 1

import numpy as np
matriz_transpose = np.transpose(matriz)
TP, TN, FP, FN = 0, 0, 0, 0
for l in range(len(matriz)):
    TP += matriz[l][l]
    TN += sum(np.diagonal(matriz)) - matriz[l][l]
    FP += sum(matriz_transpose[l]) - matriz[l][l]
    FN += sum(matriz[l]) - matriz[l][l]

TP /= len(matriz)
TN /= len(matriz)
FP /= len(matriz)
FN /= len(matriz)

In [0]:
acuracia = (TP + TN) / (TP + TN + FP + FN)
recall = TP / (TP+FN)
precisao =  TP / (TP+FP)
fscore = 2*((precisao*recall)/(precisao + recall))
print(f'Acurácia: {acuracia}, Recall: {recall}, Precisão: {precisao}, F-Score: {fscore}')

Acurácia: 0.7407407407407407, Recall: 0.4166666666666667, Precisão: 0.4166666666666667, F-Score: 0.4166666666666667


# Keras + Bert

In [0]:
%%capture
%tensorflow_version 2.x
!pip install ktrain

In [0]:
import ktrain
from ktrain import text
import pandas as pd

using Keras version: 2.2.4-tf


In [0]:
x_train, y_train, x_test, y_test = [], [], [], []
for classe in training_data:
    for sentenca in classe['sentencas']:
        y_train.append(classes.index(classe['classe']))
        x_train.append(sentenca)

for classe in test_data:
    for sentenca in classe['sentencas']:
        y_test.append(classes.index(classe['classe']))
        x_test.append(sentenca)

In [0]:
(x_train,  y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=x_train, y_train=y_train,
                                                                       x_test=x_test, y_test=y_test,
                                                                       class_names=classes,                                                                       
                                                                       preprocess_mode='bert',
                                                                       lang='pt',
                                                                       maxlen=350, 
                                                                       max_features=35000)

task: text classification
downloading pretrained BERT model (multi_cased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: pt


preprocessing test...
language: pt


In [0]:
model = text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)
learner = ktrain.get_learner(model, train_data=(x_train, y_train), batch_size=6)

Is Multi-Label? False
maxlen is 350
done.


In [0]:
learner.fit_onecycle(2e-5, 4)



begin training using onecycle policy with max lr of 2e-05...
Train on 38 samples
Epoch 1/4
38/38 [==============================] - 188s 5s/sample - loss: 2.1168 - accuracy: 0.1579
Epoch 2/4
38/38 [==============================] - 160s 4s/sample - loss: 1.9198 - accuracy: 0.3158
Epoch 3/4
38/38 [==============================] - 163s 4s/sample - loss: 1.5590 - accuracy: 0.6579
Epoch 4/4
38/38 [==============================] - 161s 4s/sample - loss: 1.2053 - accuracy: 0.7632


In [0]:
learner.validate(val_data=(x_test, y_test), class_names=classes)

               precision    recall  f1-score   support

pre_requisito       1.00      1.00      1.00         2
         sala       0.00      0.00      0.00         1
       duvida       1.00      1.00      1.00         2
      horario       0.67      1.00      0.80         2
      estagio       0.50      1.00      0.67         1
       ementa       1.00      1.00      1.00         1
carga_horária       0.00      0.00      0.00         2
    professor       0.50      1.00      0.67         1

     accuracy                           0.75        12
    macro avg       0.58      0.75      0.64        12
 weighted avg       0.61      0.75      0.66        12



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


array([[2, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 2, 0, 0, 0, 0, 0],
       [0, 0, 0, 2, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 1]])

In [0]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [0]:
predictor.save('predictor')

In [0]:
predictor.predict(test_data[3]['sentencas'][0]), test_data[3]['sentencas'][0]

('pre_requisito', 'Qual pre-requisito para prog2')

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
